In [12]:
from sklearn import datasets
from scipy import stats
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
import numpy as np

digits = datasets.load_digits()
target = digits.target
data = digits.data

scaler = StandardScaler()
data = scaler.fit_transform(data)

classifier = BaggingClassifier()

grade = {'n_estimators': [10, 25, 50, 100]}

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=4, random_state=36851234)

gridSearch = GridSearchCV(classifier, param_grid=grade,
                          scoring='accuracy', cv=3)

scoresBagging = cross_val_score(
    gridSearch, data, target, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

mean = scoresBagging.mean()
std = scoresBagging.std()
lower, upper = stats.norm.interval(0.95, loc=mean, scale=std /
                          np.sqrt(len(scoresBagging)))

print('Mean :', mean)
print('Std: ', std)
print('Accuracy Confidence Interval (95%%): (%0.2f, %0.2f)' % (lower, upper))


Mean : 0.902615470228158
Std:  0.02658335205897871
Accuracy Confidence Interval (95%): (0.87, 0.93)
